In [1]:
!nvidia-smi

Tue Jan  9 08:48:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           Off | 00000000:06:00.0 Off |                    0 |
| N/A   31C    P0              43W / 300W |      3MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU device:',torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')
    print('No GPU avaialable, Using CPU')

torch.cuda.set_device(5)

/home/sur06423/miniconda3/envs/vi_trans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU device: Tesla V100-SXM2-32GB


In [2]:
import time

def format_time(seconds):
    """Converts time in seconds to hours, minutes, and seconds format."""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds"


In [3]:
import os
import cv2
import csv
import time

class FrameExtractor:
    def __init__(self, data_dir, output_dir):
        self.data_dir = data_dir
        self.output_dir = output_dir

    def extract_frames(self, row, max_frames_per_chunk):
        participant_id, file_id, annotation_id, frame_start, frame_end, activity, chunk_id = row
        video_filepath = os.path.join(self.data_dir, file_id + '.mp4')
        new_file_id = file_id.replace("/", "_")
        updated_output_dir = os.path.join(self.output_dir, f'{activity}', f'{participant_id}_{new_file_id}_frames_{frame_start}_{frame_end}_ann_{annotation_id}_chunk_{chunk_id}')
        
        os.makedirs(updated_output_dir, exist_ok=True)
        cap = cv2.VideoCapture(video_filepath)
        frame_count = 0
         
        try:
            # Set the starting frame position
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_start))
            
            for frame_num in range(int(frame_start), int(frame_end) + 1):
                ret, frame = cap.read()
                if not ret:
                    print(f"Frame number {frame_num} is missing.")
                    break
                #new_file_id = file_id.replace("/", "_")
                output_filename = f'img_{frame_num:06d}.png'
                output_path = os.path.join(updated_output_dir, output_filename)
                cv2.imwrite(output_path, frame)
                frame_count += 1
                
                if frame_count > max_frames_per_chunk:
                    break
        finally:
            cap.release()

def process_annotations(annotation_file, data_dir, root_dataset_dir, dataset_sub_dir, max_frames_per_chunk=48):
    with open(annotation_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header row
        output_dir = os.path.join(root_dataset_dir, dataset_sub_dir)
        frame_extractor = FrameExtractor(data_dir, output_dir)
        
        for row in reader:
            frame_extractor.extract_frames(row, max_frames_per_chunk)

# First : Train : Split_1, 
def main():
    data_dir = "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color/kinect_color"
    # "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/daa_dataset"
    root_dataset_dir = "/net/polaris/storage/deeplearning/sur_data/rgb_daa/split_1"
    dataset_sub_dirs = ['val'] #, 'train', 'test', 'val'
    annotation_files = [
        '/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/kinect_color/midlevel.chunks_90.split_1.val.csv'
    ] #,
      #  '/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/kinect_color/midlevel.chunks_90.split_0.train.csv',
      #  '/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/kinect_color/midlevel.chunks_90.split_0.test.csv'
    
    # # Start timing before the loop
    start_time = time.time()

    for annotation_file, dataset_sub_dir in zip(annotation_files, dataset_sub_dirs):
        process_annotations(annotation_file, data_dir, root_dataset_dir, dataset_sub_dir)
    
    end_time = time.time()  # End timing after the loop
    time_taken = end_time - start_time  # Calculating time taken
    print(f"Time taken for extracting validation frames of split_1: {time_taken} seconds")
    formatted_time = format_time(time_taken)  # Convert to hours, minutes, seconds format
    print(f"Time taken for extracting validation frames of split_1: {formatted_time}")

#if __name__ == "__main__":
#    main()

main()

Time taken for extracting validation frames of split_1: 2262.9929809570312 seconds
Time taken for extracting validation frames of split_1: 0 hours, 37 minutes, 42 seconds
